# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,356.892838,-0.072763,1234,356.820076,356.965601,713.640151,713.931201
6,289.021944,-0.961860,1234,288.060084,289.983803,576.120168,579.967607
7,96.493321,0.883889,1234,97.377210,95.609432,194.754420,191.218865
8,382.772984,0.736674,1234,383.509659,382.036310,767.019317,764.072620
9,135.025906,0.947412,1234,135.973318,134.078493,271.946636,268.156987
10,592.589458,-0.796391,1234,591.793067,593.385849,1183.586134,1186.771698
11,966.375037,-0.353350,1234,966.021688,966.728387,1932.043375,1933.456774
12,244.504387,-0.095270,1234,244.409117,244.599657,488.818234,489.199314
13,161.764772,0.833528,1234,162.598301,160.931244,325.196602,321.862488
14,815.052291,-0.419709,1234,814.632583,815.472000,1629.265166,1630.944000


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-356.820076,356.965601
6,-288.060084,289.983803
7,-97.377210,95.609432
8,-383.509659,382.036310
9,-135.973318,134.078493
10,-591.793067,593.385849
11,-966.021688,966.728387
12,-244.409117,244.599657
13,-162.598301,160.931244
14,-814.632583,815.472000


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-356.820076,356.965601
6,-288.060084,289.983803
7,-97.377210,95.609432
8,-383.509659,382.036310
9,-135.973318,134.078493
10,-591.793067,593.385849
11,-966.021688,966.728387
12,-244.409117,244.599657
13,-162.598301,160.931244
14,-814.632583,815.472000


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-356.82007554,  356.9656006 ],
       [-288.06008409,  289.98380348],
       [ -97.37720988,   95.60943226],
       [-383.50965853,  382.03631012],
       [-135.97331778,  134.07849349],
       [-591.7930672 ,  593.38584878],
       [-966.02168765,  966.72838723],
       [-244.40911709,  244.59965705],
       [-162.59830081,  160.93124392],
       [-814.63258293,  815.47199997]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,711.640151,-713.931201
6,574.120168,-579.967607
7,192.754420,-191.218865
8,765.019317,-764.072620
9,269.946636,-268.156987
10,1181.586134,-1186.771698
11,1930.043375,-1933.456774
12,486.818234,-489.199314
13,323.196602,-321.862488
14,1627.265166,-1630.944000


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
